This notebook was created in google colab
https://colab.research.google.com/



In [ ]:
# Step 1: Setting Up Google Colab and Installing Dependencies
!pip install langchain
!pip install openai
!pip install chromadb
!pip install unstructured
!pip install pdfplumber
!pip install google-search-results
!pip install langchain-community
!pip install langchain-openai

# Install python-docx (if not already installed):
!pip install python-docx

# Make sure SerpAPI is installed:
!pip install google-search-results

# Install Dependencies (if needed)
!pip install reportlab

Set Up API Keys in Colab



In [ ]:
import os

# Set API keys securely
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""

Upload the Resume File in Google Colab


In [ ]:
from google.colab import files

uploaded = files.upload()  # Uploads the file
resume_path = list(uploaded.keys())[0]  # Gets the file name
print(f"Uploaded file: {resume_path}")

Extract Text from the Resume


In [ ]:
# Extract the text

import pdfplumber
import docx

def extract_text_from_resume(file_path):
    if file_path.lower().endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    elif file_path.lower().endswith(".docx"):
        doc = docx.Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text

resume_text = extract_text_from_resume(resume_path)
print("Extracted Resume Text (First 500 chars):\n", resume_text[:500])  # Preview

Step 4: Analyze Skills and Competencies from the Resume

In [5]:
#from langchain.chat_models import ChatOpenAI
#from langchain.prompts import PromptTemplate
#from langchain.chains import LLMChain

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Initialize OpenAI model
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["resume_text"],
    template="""Extract the following details from the given resume text:
    - List of key skills (technical & soft skills)
    - Work experiences (company, position, duration)
    - Education details (degree, institution, year)
    - Certifications (if any)
    - Summary of expertise

    Resume Text:
    {resume_text}

    Provide the extracted details in a structured format.
    """
)

# Create the chain
resume_analysis_chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain on the extracted resume text
analysis_result = resume_analysis_chain.run(resume_text)

print("Extracted Resume Details:\n", analysis_result)

<ipython-input-5-14ff8e5473d0>:31: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  resume_analysis_chain = LLMChain(llm=llm, prompt=prompt_template)
<ipython-input-5-14ff8e5473d0>:34: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  analysis_result = resume_analysis_chain.run(resume_text)


Extracted Resume Details:
 - Key Skills:
    - Technical: Project Management, Software Engineering, Digital Transformation, C# Programming, Azure resource administration, Agile Methodologies, Quality Management, Risk Management, Auditing
    - Soft Skills: Cross-location collaboration

- Work Experiences:
    - Senior Product Manager at TripleCyber, from Jan 2022 to Jan 2025
    - Project Manager at Agilisa Technologies, from Mar 2020 to Dec 2021
    - Business Advisor at CMTD-ITLA, from Mar 2019 to Feb 2020
    - Project Manager at SENASA, from May 2012 to Jul 2018

- Education Details:
    - Master in Business Administration from UASD, Dominican Republic in Nov 2019
    - System Engineer from O&M University, Dominican Republic in Jan 2008
    - PMP (2236059) from Project Management Institute in Jul 2018

- Certifications:
    - PMP (2236059) from Project Management Institute in Jul 2018

- Summary of Expertise:
    - Results-driven Project Manager with 20+ years of experience in the 

Then, import the necessary libraries:

In [6]:
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper

# Initialize SerpAPI for job search
search = SerpAPIWrapper()

Create a Function to Search for Jobs


In [ ]:
def search_jobs(query, location="remote", num_results=5):
    search_query = f"{query} jobs in {location}"

    # results = search.results(search_query, num_results)
    results = search.results(query=search_query)

    print(f"Search results for '{search_query}':")
    print(results)

    job_listings = []
    for result in results["organic_results"]:
        job_listings.append({
            "title": result["title"],
            "company": result["source"],
            "link": result["link"],
            "description": result["snippet"]
        })

    return job_listings

# Example: Search for jobs using extracted skills
skills_query = analysis_result  # Using the extracted skills from the resume
jobs = search_jobs(skills_query)

# Display job results
for idx, job in enumerate(jobs, start=1):
    print(f"{idx}. {job['title']} at {job['company']}")
    print(f"Link: {job['link']}\n")

Step 6: Customize the Resume for a Job Listing


In [8]:
# 2. Generate a Tailored Resume
def generate_tailored_resume(resume_text, job_title, company, job_description):


    # Define a prompt template for resume optimization
    resume_prompt = PromptTemplate(
        input_variables=["resume_text", "job_title", "company", "job_description"],
        template=f"""Based on the given resume, tailor a new version that highlights
        relevant skills and experience for the following job:

        Job Title: {job_title}
        Company: {company}
        Job Description: {job_description}

        Original Resume:
        {resume_text}

        Generate a new, optimized resume for submission.
        """
    )

    # Create a LangChain chain
    resume_chain = LLMChain(llm=llm, prompt=resume_prompt)

    # Simulated job description (if not available in search results)
    job_description = "We are looking for a skilled software developer with experience in c#, ci/cd, and cloud computing."

    # Generate the tailored resume
    tailored_resume = resume_chain.run({
        "resume_text": resume_text,
        "job_title": job_title,
        "company": company,
        "job_description": job_description
    })

    return tailored_resume

print("done")

done


Step 7: Save the Tailored Resume as a PDF or DOCX


In [10]:
from docx import Document

def save_resume_as_docx(text, filename="tailored_resume.docx"):
    doc = Document()
    doc.add_paragraph(text)
    doc.save(filename)
    print(f"Resume saved as {filename}")

# Save the tailored resume as a Word document
# save_resume_as_docx(tailored_resume)

In [11]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def save_resume_as_pdf(text, filename="tailored_resume.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    # Split text into lines and write to PDF
    lines = text.split("\n")
    y_position = height - 40  # Start position

    for line in lines:
        c.drawString(40, y_position, line)
        y_position -= 15  # Move down for next line

    c.save()
    print(f"Resume saved as {filename}")

# Save the tailored resume as a PDF
# save_resume_as_pdf(tailored_resume)

In [12]:
import sys
import datetime
import re

def generate_structured_filename(index, company_name):
    """
    Generates a filename with the structure: {index} - {company name} - {YYYY_MM_DD_HH_MM}.pdf

    Args:
        index (str): The index number or identifier.
        company_name (str): The name of the company.

    Returns:
        str: The generated filename string.
    """
    # Get current date and time
    now = datetime.datetime.now()
    timestamp_str = now.strftime("%Y_%m_%d_%H_%M")

    # Clean company name for filename (replace spaces with underscores, remove special chars)
    cleaned_company_name = re.sub(r'[^a-zA-Z0-9\s]', '', company_name).strip().replace(' ', '_')

    # Construct the filename
    filename = f"{index} - {cleaned_company_name} - {timestamp_str}.pdf"
    return filename

In [13]:
# iterar the result
for idx, job in enumerate(jobs, start=1):
    print(f"{idx}. {job['title']} at {job['company']}")
    print(f"Link: {job['link']}\n")
    print(f"Description: {job['description']}\n")

    index = idx
    company_name = job['company']
    job_title = job['title']
    job_description = job['description']

    filename = generate_structured_filename(index, company_name)
    print(f"Generated filename: {filename}")

    #filename = generate_structured_filename(idx, job['company'])
    #save_resume_as_pdf(tailored_resume, filename)
    #print(f"Resume saved as {filename}")

    tailored_resume = generate_tailored_resume(resume_text, job_title, company_name, job_description)
    save_resume_as_pdf(tailored_resume, f"{filename}.pdf")


    print("=" * 50)




1. Software/IT Project Managers - key skills required? at Reddit · r/projectmanagement
Link: https://www.reddit.com/r/projectmanagement/comments/jkauab/softwareit_project_managers_key_skills_required/

Description: Software/IT Project Managers - key skills required? · PM software: Microsoft Project, Smartsheet, Jira · Languages/tools: SQL, Python, GitHub.

Generated filename: 1 - Reddit__rprojectmanagement - 2025_02_19_21_43.pdf
Resume saved as 1 - Reddit__rprojectmanagement - 2025_02_19_21_43.pdf.pdf
2. What are the top skills for an agile software team leader? at LinkedIn
Link: https://www.linkedin.com/advice/1/what-top-skills-agile-software-team-ntllc

Description: Learn how to communicate, coach, facilitate, lead, adapt, and contribute as an agile software team leader and deliver successful software ...

Generated filename: 2 - LinkedIn - 2025_02_19_21_43.pdf
Resume saved as 2 - LinkedIn - 2025_02_19_21_43.pdf.pdf
3. Vishal M. - Sr Technical Recruiter - Pyramid Consulting, Inc at L